## Генерируем train выборки
В данном ноутбуке класстеризуем изображения и выбираем необходимые для конкретной задачи обучения.
По большей части ноутбук является устаревшим, т.к. класстеризация была усовершеноствованна.



## Создание выборки из кораблей
Выборка для обучения на средних и маленьких коряблях
Изображения не содеражат пейзажей и портов. Фон однотонный + изображения без кораблей с ребристым фоном - гребни волн.

In [2]:
from PIL import Image
from scipy import ndimage
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import numpy as np
import matplotlib.pyplot as plt
import os
import time
#TEST

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    if isinstance(mask_rle, float):
        return 0
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


In [3]:
masks = pd.read_csv('F:\\Downloads\\all\\train_ship_segmentations_v2.csv')
masks.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)

masks.head()

,ImageId,EncodedPixels,ships
0,00003e153.jpg,NaN,0
1,0001124c7.jpg,NaN,0
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,1
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...,1
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...,1


In [7]:
# список изображений без кораблей
masks_none = []
for i in range(masks.shape[0]):
    if masks.ships[i] == 0:
        masks_none.append(masks.ImageId[i])

In [8]:
print(len(masks_none))
print(masks_none[58259])
ERROR_FILE = '6384c3e78.jpg'

150000
6384c3e78.jpg


### Найдем пустые изображения с гребными на волнах

In [7]:
for i in range(135000, len(masks_none)):
    if i % 1000 == 0:
        print('Processing {} thousand images'.format(i // 1000))
    c_img = masks_none[i]
    if c_img == ERROR_FILE:
        continue
    img = Image.open('F:\\Downloads\\all\\train_v2\\{}'.format(c_img))
    img_bin = img.convert('L')
    img_bin = np.array(img_bin)

    ca = np.mean(img_bin)
    cv = np.mean(np.abs(img_bin - ca))

    y = (np.abs(img_bin - ca) > 50) * 1.0
    sBin = np.sum(y)/(768*768)
    
    img_masks = masks.loc[masks['ImageId'] == c_img, 'EncodedPixels'].tolist()
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768))
    for mask in img_masks:
        all_masks += rle_decode(mask)
    
    labels, n_labels = ndimage.label(all_masks)
    
    if sBin < 0.07 and cv < 12 and cv > 10 and n_labels < 4:
        img.save('F:\\Downloads\\all\\monotone_waves_v2\\{}'.format(c_img))

Processing 135 thousand images
Processing 136 thousand images
Processing 137 thousand images
Processing 138 thousand images
Processing 139 thousand images
Processing 140 thousand images
Processing 141 thousand images
Processing 142 thousand images
Processing 143 thousand images
Processing 144 thousand images
Processing 145 thousand images
Processing 146 thousand images
Processing 147 thousand images
Processing 148 thousand images
Processing 149 thousand images


In [269]:
i

60485

## Блок для проверки

In [261]:
#c_img = '002c872c1.jpg'
c_img = '00f24b45c.jpg'
img = Image.open('F:\\Downloads\\all\\train_v2\\{}'.format(c_img))
img_rgb = np.array(img)
print(np.sum(img_rgb.shape[2])/(768*768))



img_bin = img.convert('L')
img_bin = np.array(img_bin)

ca = np.mean(img_bin)
cv = np.mean(np.abs(img_bin - ca))

y = (np.abs(img_bin - ca) > 30) * 1.0
#plt.imshow(y)
sBin = np.sum(y)/(768*768)
print(sBin, cv)

5.086263020833333e-06
0.04554578993055555 11.967275233203427


## Создание датасета с мелкими и средними кораблями + следы за кораблями 

In [10]:
%%time

minS = 500
maxS = 8000

images = os.listdir('F:\\Downloads\\all\\train_v2\\')
print(len(images))
start = time.time()
for i in range(5):
    if i % 1000 == 0:
        print('Processing {} thousands images'.format(i // 1000), 
              time.time() - start)
        start = time.time()
    if images[i] == ERROR_FILE:
        continue
    img = Image.open('F:\\Downloads\\all\\train_v2\\{}'.format(images[i]))
    img_data = np.asarray(img.convert('L'))

    ca = np.mean(img_data)
    cv = np.mean(np.abs(img_data - ca))
    y = (np.abs(img_data - ca) > 30) * 1.0
    sBin = np.sum(y)/(768*768)
    
    img_masks = masks.loc[masks['ImageId'] == images[i], 'EncodedPixels'].tolist()
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768))
    print(img_masks)
    for mask in img_masks:
        all_masks += rle_decode(mask)
    
    labels, n_labels = ndimage.label(all_masks)
    if n_labels < 4 and cv < 5 and sBin < 0.003:
        param = np.zeros(n_labels)

        for j in np.arange(1, n_labels+1):
                square = np.where(labels == j)               
                if len(square[0]) < minS:
                    param[j-1] = 1
                else:
                    break
        summar = np.sum(param)
        
        if summar > 0:
            img.save('F:\\Downloads\\all\\{}'.format(images[i]))

192556
Processing 0 thousands images 0.0
[nan]
[nan]
['264661 17 265429 33 266197 33 266965 33 267733 33 268501 33 269269 33 270037 33 270805 33 271573 33 272341 33 273109 33 273877 33 274645 33 275413 33 276181 33 276949 33 277716 34 278484 34 279252 33 280020 33 280788 33 281556 33 282324 33 283092 33 283860 33 284628 33 285396 33 286164 33 286932 33 287700 33 288468 33 289236 33 290004 33 290772 33 291540 33 292308 33 293076 33 293844 33 294612 33 295380 33 296148 33 296916 33 297684 33 298452 33 299220 33 299988 33 300756 33 301524 33 302292 33 303060 33 303827 34 304595 34 305363 33 306131 33 306899 33 307667 33 308435 33 309203 33 309971 33 310739 33 311507 33 312275 33 313043 33 313811 33 314579 33 315347 33 316115 33 316883 33 317651 33 318419 33 319187 33 319955 33 320723 33 321491 33 322259 33 323027 33 323795 33 324563 33 325331 33 326099 33 326867 33 327635 33 328403 33 329171 33 329938 34 330706 34 331474 33 332242 33 333010 33 333778 33 334546 33 335314 33 336082 33 33685

In [289]:
i

74846

In [7]:
import pandas as pd
import numpy as np
import os

In [23]:
f = open('F:\\Downloads\\all\\motley.txt', 'r')

In [24]:
images_names = []
for line in f:
    images_names.append(line[:len(line)-1])

In [25]:
len(images_names)

17238

In [26]:
masks = pd.read_csv(os.path.join('F:\\Downloads\\all\\',
                                 'train_ship_segmentations_v2.csv'))

In [27]:
%%time
data = masks.values
new_data = []

for i in range(len(images_names)):
    if i % 10000 == 0:
        print('Processing {} thousand images'.format(i // 1000))
    c_img = images_names[i]
    rows, _ = np.where(data == c_img)
    for j in range(data[rows].shape[0]):
        new_data.append(data[rows][j])

Processing 0 thousand images
Processing 10 thousand images
Wall time: 4min 11s


In [28]:
data_csv = pd.DataFrame(new_data, columns=['ImageId', 'EncodedPixels'])
name_file = 'motley_train.csv'
data_csv.to_csv('F:\\Downloads\\all\\' + name_file)

In [32]:
from PIL import Image

images_names = pd.read_csv('F:\\Downloads\\all\\harbors_train.csv').ImageId
print(len(images_names))

for i in range(len(images_names)):
    if i % 1000 == 0:
        print('Processing {} thousand images'.format(i // 1000))
    c_img = images_names[i]
    img = Image.open('F:\\Downloads\\all\\train_v2\\{}'.format(c_img))    
    img.save('F:\\Downloads\\all\\train_harbors\\{}'.format(c_img))

64156
Processing 0 thousand images
Processing 1 thousand images
Processing 2 thousand images
Processing 3 thousand images
Processing 4 thousand images
Processing 5 thousand images
Processing 6 thousand images
Processing 7 thousand images
Processing 8 thousand images
Processing 9 thousand images
Processing 10 thousand images
Processing 11 thousand images
Processing 12 thousand images
Processing 13 thousand images
Processing 14 thousand images
Processing 15 thousand images
Processing 16 thousand images
Processing 17 thousand images
Processing 18 thousand images
Processing 19 thousand images
Processing 20 thousand images
Processing 21 thousand images
Processing 22 thousand images
Processing 23 thousand images
Processing 24 thousand images
Processing 25 thousand images
Processing 26 thousand images
Processing 27 thousand images
Processing 28 thousand images
Processing 29 thousand images
Processing 30 thousand images
Processing 31 thousand images
Processing 32 thousand images
Processing 33 